====== Nguồn http://users.soict.hust.edu.vn/khoattq/ml-dm-course/ ======



## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files


%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [5]:
INPUT = 'data/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh kết quả trong quá trình huấn luyện và đánh giá

In [7]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 180
-------------------------
Tổng số văn bản: 354


In [15]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
--------------------------
['data/news_vnexpress\\doi-song\\00006.txt']
[0]
['Nhân ngày lễ tình nhân 14/2, Viktoria Pustovitova, 28 tuổi và Alexander Kudlay, 33 tuổi, quyết định xích tay nhau ba tháng để thử thách tình yêu.Sau khi "chung xích", đôi tình nhân ở Kharkiv tin rằng họ có thể cùng nhau vượt qua mọi sự bất tiện trong sinh hoạt, từ rửa bát, nấu thức ăn đến đi vệ sinh và lái xe ô tô.Tuy nhiên, hôm 26/2, truyền thông Ukraine đưa tin, sợi xích đã khiến cô Viktoria bị loét cổ tay. "Chúng tôi đã bôi thuốc nhưng không được. Đó quả là một cơn ác mộng", cô gái nói.Họ buộc phải đến bệnh viện để xử lý vết thương ở cổ tay.Bác sĩ da liễu Tatyana Egorova cảnh báo, đôi tình nhân có thể phải từ bỏ thử thách tình yêu. "Đây là vết loét do tiếp xúc lâu với kim loại. Các mô da bị kẹp liên tục nên dẫn đến rối loạn tuần hoàn", bác sĩ này nói.Viktoria và Alexander có thể làm mọi việc hàng ngày cùng nhau. Tuy nhiên, vào nhà vệ sinh công cộng khiến họ gặ

## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

In [22]:
# load dữ liệu các stopwords 
with open("data/vietnamese-stopwords.txt", encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords] 
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])

# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng
#     - sinh từ điển
module_count_vector = CountVectorizer(stop_words=stopwords)
model_rf_preprocess = Pipeline([('vect', module_count_vector),
                    ('tfidf', TfidfTransformer()),
                    ])
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data, data_train.target)

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {data_preprocessed.shape}")
print(f"Kích thước nhãn tương ứng: {data_train.target.shape}")

Số lượng stopwords: 2063
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 6413
Kích thước dữ liệu sau khi xử lý: (354, 6413)
Kích thước nhãn tương ứng: (354,)


In [24]:
X = data_preprocessed
Y = data_train.target

In [26]:
X.shape, Y.shape

((354, 6413), (354,))

In [28]:
print(X[100].toarray())
print(Y[100])

[[0. 0. 0. ... 0. 0. 0.]]
2


In [30]:
sum(sum(X[100].toarray() != 0))

5

In [32]:
print(X[100])

  (0, 6384)	0.18895921086566383
  (0, 5791)	0.3905893510067745
  (0, 2451)	0.5034928675157354
  (0, 1708)	0.6953314500077334
  (0, 442)	0.2733925428757287
